<a href="https://colab.research.google.com/github/ashishpatel26/tcn-keras-Examples/blob/master/TCN_Many_to_Many_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install keras-tcn
!pip install wget
from tcn import compiled_tcn
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import wget

In [5]:
wget.download('https://github.com/philipperemy/keras-tcn/raw/master/tasks/exchange_rate/exchange_rate.txt')

'exchange_rate (1).txt'

### Read data

In [16]:
def get_xy_kfolds(split_index=[0.5, 0.6, 0.7, 0.8, 0.9], timesteps=1000):
    """
    load exchange rate dataset and preprecess it, then split it into k-folds for CV
    :param split_index: list, the ratio of whole dataset as train set
    :param timesteps: length of a single train x sample
    :return: list, [train_x_set,train_y_set,test_x_single,test_y_single]
    """
    df = np.loadtxt('exchange_rate.txt', delimiter=',')
    n = len(df)
    folds = []
    enc = MinMaxScaler()
    df = enc.fit_transform(df)
    for split_point in split_index:
        train_end = int(split_point * n)
        train_x, train_y = [], []
        for i in range(train_end - timesteps):
            train_x.append(df[i:i + timesteps])
            train_y.append(df[i + timesteps])
        train_x = np.array(train_x)
        train_y = np.array(train_y)
        test_x = df[train_end - timesteps + 1:train_end + 1]
        test_y = df[train_end + 1]
        folds.append((train_x, train_y, test_x, test_y))
    return folds, enc

### Folds Contrains 5 Value with List below
* Change the folds value by just passing like below example

```
We have 5 Folds Values

  train_x, train_y, test_x, test_y = folds[0]
  train_x, train_y, test_x, test_y = folds[1]
  train_x, train_y, test_x, test_y = folds[2]
  train_x, train_y, test_x, test_y = folds[3]
  train_x, train_y, test_x, test_y = folds[4]
```

In [28]:
folds, enc = get_xy_kfolds(split_index=[0.5, 0.6, 0.7, 0.8, 0.9], timesteps=1000)

mse_list = []
# train_x, train_y, test_x, test_y = folds[0]
train_x, train_y, test_x, test_y = folds[1]
# train_x, train_y, test_x, test_y = folds[2]
# train_x, train_y, test_x, test_y = folds[3]
# train_x, train_y, test_x, test_y = folds[4]
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(3552, 1000, 8) (3552, 8) (1000, 8) (8,)


### Model Design

In [29]:
#model object has defined
model = compiled_tcn(return_sequences=False,
                      num_feat=test_x.shape[1],
                      nb_filters=24,
                      num_classes=0,
                      kernel_size=8,
                      dilations=[2 ** i for i in range(9)],
                      nb_stacks=1,
                      max_len=test_x.shape[0],
                      use_skip_connections=True,
                      regression=True,
                      dropout_rate=0,
                      output_len=test_y.shape[0])

model.summary()

x.shape= (None, 24)
model.x = (None, 1000, 8)
model.y = (None, 8)
Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1000, 8)]         0         
_________________________________________________________________
tcn (TCN)                    (None, 24)                80520     
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 200       
_________________________________________________________________
activation_3 (Activation)    (None, 8)                 0         
Total params: 80,720
Trainable params: 80,720
Non-trainable params: 0
_________________________________________________________________


### training the model

In [30]:
# training the model
model.fit(train_x, train_y, batch_size=256, epochs=30)

Epoch 1/30
14/14 [==============================] - 3s 236ms/step - loss: 29.2627
Epoch 2/30
14/14 [==============================] - 3s 232ms/step - loss: 0.0192
Epoch 3/30
14/14 [==============================] - 3s 233ms/step - loss: 0.0085
Epoch 4/30
14/14 [==============================] - 3s 234ms/step - loss: 0.0050
Epoch 5/30
14/14 [==============================] - 3s 235ms/step - loss: 0.0037
Epoch 6/30
14/14 [==============================] - 3s 234ms/step - loss: 0.0030
Epoch 7/30
14/14 [==============================] - 3s 233ms/step - loss: 0.0025
Epoch 8/30
14/14 [==============================] - 3s 231ms/step - loss: 0.0022
Epoch 9/30
14/14 [==============================] - 3s 230ms/step - loss: 0.0020
Epoch 10/30
14/14 [==============================] - 3s 229ms/step - loss: 0.0018
Epoch 11/30
14/14 [==============================] - 3s 229ms/step - loss: 0.0017
Epoch 12/30
14/14 [==============================] - 3s 228ms/step - loss: 0.0016
Epoch 13/30
14/14 [=====

### Predict the X_test Value

In [25]:
# predict the value from x_test
y_raw_pred = model.predict(np.array([test_x]))

# Invert transform for get a original value
y_pred = enc.inverse_transform(y_raw_pred).flatten()
y_true = enc.inverse_transform([test_y]).flatten()

### Model Evaluation with Mean Square Error

In [27]:
# Calculate the Mean Square Error for Regression Problem
mse_cur = mean_squared_error(y_true, y_pred)

# Append the error in to mse_list
# mse_list.append(mse_cur)

# print the shape of xtrain
# print(f"train_set_size:{train_x.shape[0]}")

# print the y_true and y_pred
# print(f"y_true:{y_true}")
# print(f"y_pred:{y_pred}")

# print the current MSE value
print(f"mse:{mse_cur}")

# print(f"finial loss on test set: {np.mean(mse_list)}")

mse:0.00032251839564405055
